# Initialising the code

In [ ]:
import csv
from pathlib import Path
import numpy as np
from ranking import Ranking
from index import Mode, InMemoryIndex
from encoder import TCTColBERTQueryEncoder as TCTColBERTQueryEncoderFF
# from pyserini.search.faiss import FaissSearcher, TctColBertQueryEncoder
from ir_measures import read_trec_qrels, calc_aggregate, nDCG, RR, P, R, AP, MAP
import sys
from tqdm import tqdm

# define values of eta to test 
etas = [10,20,30,40,50,60,70,80,90,100,500]

# define values of alpha to test
alphas = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,-1.0]

In [ ]:
# in_memory_index = InMemoryIndex(mode=Mode.PASSAGE, encoder=TCTColBERTQueryEncoderFF("castorini/tct_colbert-msmarco"))
in_memory_index = InMemoryIndex.from_disk('../testing/ffindex_passage_2019_2020.pkl')
in_memory_index.encoder = TCTColBERTQueryEncoderFF("castorini/tct_colbert-msmarco")

In [ ]:
sparse_ranking_2019 = Ranking.from_file(Path("../testing/msmarco-passage-test2019-sparse10000.txt"))
sparse_ranking_2019.cut(5000)
sparse_ranking_2019.name = "sparse_ranking_2019"
sparse_ranking_2020 = Ranking.from_file(Path("../testing/msmarco-passage-test2020-sparse10000.txt"))
sparse_ranking_2020.cut(5000)
sparse_ranking_2020.name = "sparse_ranking_2020"
all_ids = set.union(
    *[set(sparse_ranking_2019[q_id].keys()) for q_id in sparse_ranking_2019.q_ids],
    *[set(sparse_ranking_2020[q_id].keys()) for q_id in sparse_ranking_2020.q_ids]
)
print(f"indexing {len(all_ids)} documents or passages")

## Testing the implemented retrieval functions

In [ ]:
with open(
    "../testing/msmarco-test2019-queries.tsv",
    encoding="utf-8",
    newline=""
) as fp:
    queries = {q_id: q for q_id, q in csv.reader(fp, delimiter="\t")}
print(f"loaded {len(queries)} queries")

alpha = 0.2
result = in_memory_index.get_scores(
    sparse_ranking_2019,
    queries,
    alpha=alpha,
    cutoff=None,
    early_stopping=False,
    normalise=False
)
result2 = in_memory_index.get_scores_rrf(
    sparse_ranking_2019,
    queries,
    alpha=alpha,
    cutoff=None,
    early_stopping=False,
)
result3 = in_memory_index.get_scores_rrf(
    sparse_ranking_2019,
    queries,
    alpha=alpha,
    cutoff=None,
    early_stopping=False,
    eta=60,
    eta2=10,
)
qrels = list(read_trec_qrels("../testing/2019qrels-pass.txt"))
print(
    "BM25",
    calc_aggregate([nDCG@10, RR(rel=2)@10], qrels, sparse_ranking_2019.run)
)
print(
    f"Interpolation",
    calc_aggregate([nDCG@10, RR(rel=2)@10], qrels, result[alpha].run)
)
print(
    f"RRF",
    calc_aggregate([nDCG@10, RR(rel=2)@10], qrels, result2[alpha].run)
)
print(
    f"rrf with eta_lex = 60 and eta_sem = 10",
    calc_aggregate([nDCG@10, RR(rel=2)@10], qrels, result3[alpha].run)
)

# Validation on 2019 set

## Validation on the convex combination with two types of normalisation

In [ ]:
with open(
    "../testing/msmarco-test2019-queries.tsv",
    encoding="utf-8",
    newline=""
) as fp:
    queries = {q_id: q for q_id, q in csv.reader(fp, delimiter="\t")}
print(f"loaded {len(queries)} queries")

# define dictionary to keep track of best results for tested hyperparameters
bestHyperparams2019 = {
    'ccUnnormalized': {'precision': {'value': -1, 'alpha': 0}, 'recall': {'value': -1, 'alpha': 0}, 'fscore': {'value': -1, 'alpha': 0}, 'rr': {'value': -1, 
'alpha': 0}, 'ndcg': {'value': -1, 'alpha': 0}, 'map': {'value': -1, 'alpha': 0}},
    'ccNormalized_minimax': {'precision': {'value': -1, 'alpha': 0}, 'recall': {'value': -1, 'alpha': 0}, 'fscore': {'value': -1, 'alpha': 0}, 'rr': {'value': -1, 
'alpha': 0}, 'ndcg': {'value': -1, 'alpha': 0}, 'map': {'value': -1, 'alpha': 0}},
'ccNormalized_zscore': {'precision': {'value': -1, 'alpha': 0}, 'recall': {'value': -1, 'alpha': 0}, 'fscore': {'value': -1, 'alpha': 0}, 'rr': {'value': -1, 
'alpha': 0}, 'ndcg': {'value': -1, 'alpha': 0}, 'map': {'value': -1, 'alpha': 0}},
}


for i in tqdm(range(len(etas))):
    alpha = alphas[i]
    eta = etas[i]

    result = in_memory_index.get_scores(
        sparse_ranking_2019,
        queries,
        alpha=alpha,
        cutoff=None,
        early_stopping=False,
        normalise="none"
    )
    result2 = in_memory_index.get_scores(
        sparse_ranking_2019,
        queries,
        alpha=alpha,
        cutoff=None,
        early_stopping=False,
        normalise="minimax"
    )
    result3 = in_memory_index.get_scores(
        sparse_ranking_2019,
        queries,
        alpha=alpha,
        cutoff=None,
        early_stopping=False,
        normalise="zscore"
    )
    qrels = list(read_trec_qrels("../testing/2019qrels-pass.txt"))

    interpolationRes = calc_aggregate([nDCG@10, RR(rel=2)@10,P(rel=2)@10,R(rel=2)@10, AP(rel=2)@10], qrels, result[alpha].run)
    fscoreInterpolationRes = (2*interpolationRes[P(rel=2)@10]*interpolationRes[R(rel=2)@10])/(interpolationRes[P(rel=2)@10]+interpolationRes[R(rel=2)@10])

    interpolationMinMax = calc_aggregate([nDCG@10, RR(rel=2)@10,P(rel=2)@10,R(rel=2)@10, AP(rel=2)@10], qrels, result2[alpha].run)
    fscoreInterpolationMinMax = (2*interpolationMinMax[P(rel=2)@10]*interpolationMinMax[R(rel=2)@10])/(interpolationMinMax[P(rel=2)@10]+interpolationMinMax[R(rel=2)@10])

    interpolationZscore = calc_aggregate([nDCG@10, RR(rel=2)@10,P(rel=2)@10,R(rel=2)@10, AP(rel=2)@10], qrels, result3[alpha].run)
    fscoreInterpolationZscore = (2*interpolationZscore[P(rel=2)@10]*interpolationZscore[R(rel=2)@10])/(interpolationZscore[P(rel=2)@10]+interpolationZscore[R(rel=2)@10])

    # update dictionary of best parameters per method
    if interpolationRes[P(rel=2)@10] > bestHyperparams2019['ccUnnormalized']['precision']['value']:
        bestHyperparams2019['ccUnnormalized']['precision']['value'] = interpolationRes[P(rel=2)@10]
        bestHyperparams2019['ccUnnormalized']['precision']['alpha'] = alpha

    if interpolationRes[R(rel=2)@10] > bestHyperparams2019['ccUnnormalized']['recall']['value']:
        bestHyperparams2019['ccUnnormalized']['recall']['value'] = interpolationRes[R(rel=2)@10]
        bestHyperparams2019['ccUnnormalized']['recall']['alpha'] = alpha

    if interpolationRes[nDCG@10] > bestHyperparams2019['ccUnnormalized']['ndcg']['value']:
        bestHyperparams2019['ccUnnormalized']['ndcg']['value'] = interpolationRes[nDCG@10]
        bestHyperparams2019['ccUnnormalized']['ndcg']['alpha'] = alpha

    if interpolationRes[RR(rel=2)@10] > bestHyperparams2019['ccUnnormalized']['rr']['value']:
        bestHyperparams2019['ccUnnormalized']['rr']['value'] = interpolationRes[RR(rel=2)@10]
        bestHyperparams2019['ccUnnormalized']['rr']['alpha'] = alpha

    if interpolationRes[AP(rel=2)@10] > bestHyperparams2019['ccUnnormalized']['map']['value']:
        bestHyperparams2019['ccUnnormalized']['map']['value'] = interpolationRes[AP(rel=2)@10]
        bestHyperparams2019['ccUnnormalized']['map']['alpha'] = alpha
    
    if fscoreInterpolationRes > bestHyperparams2019['ccUnnormalized']['fscore']['value']:
        bestHyperparams2019['ccUnnormalized']['fscore']['value'] = fscoreInterpolationRes
        bestHyperparams2019['ccUnnormalized']['fscore']['alpha'] = alpha
    
    if interpolationMinMax[P(rel=2)@10] > bestHyperparams2019['ccNormalized_minimax']['precision']['value']:
        bestHyperparams2019['ccNormalized_minimax']['precision']['value'] = interpolationMinMax[P(rel=2)@10]
        bestHyperparams2019['ccNormalized_minimax']['precision']['alpha'] = alpha

    if interpolationMinMax[R(rel=2)@10] > bestHyperparams2019['ccNormalized_minimax']['recall']['value']:
        bestHyperparams2019['ccNormalized_minimax']['recall']['value'] = interpolationMinMax[R(rel=2)@10]
        bestHyperparams2019['ccNormalized_minimax']['recall']['alpha'] = alpha

    if interpolationMinMax[nDCG@10] > bestHyperparams2019['ccNormalized_minimax']['ndcg']['value']:
        bestHyperparams2019['ccNormalized_minimax']['ndcg']['value'] = interpolationMinMax[nDCG@10]
        bestHyperparams2019['ccNormalized_minimax']['ndcg']['alpha'] = alpha

    if interpolationMinMax[RR(rel=2)@10] > bestHyperparams2019['ccNormalized_minimax']['rr']['value']:
        bestHyperparams2019['ccNormalized_minimax']['rr']['value'] = interpolationMinMax[RR(rel=2)@10]
        bestHyperparams2019['ccNormalized_minimax']['rr']['alpha'] = alpha
    
    if fscoreInterpolationMinMax > bestHyperparams2019['ccNormalized_minimax']['fscore']['value']:
        bestHyperparams2019['ccNormalized_minimax']['fscore']['value'] = fscoreInterpolationMinMax
        bestHyperparams2019['ccNormalized_minimax']['fscore']['alpha'] = alpha
    
    if interpolationMinMax[AP(rel=2)@10] > bestHyperparams2019['ccNormalized_minimax']['map']['value']:
        bestHyperparams2019['ccNormalized_minimax']['map']['value'] = interpolationMinMax[AP(rel=2)@10]
        bestHyperparams2019['ccNormalized_minimax']['map']['alpha'] = alpha

    if interpolationMinMax[P(rel=2)@10] > bestHyperparams2019['ccNormalized_zscore']['precision']['value']:
        bestHyperparams2019['ccNormalized_zscore']['precision']['value'] = interpolationZscore[P(rel=2)@10]
        bestHyperparams2019['ccNormalized_zscore']['precision']['alpha'] = alpha

    if interpolationZscore[R(rel=2)@10] > bestHyperparams2019['ccNormalized_zscore']['recall']['value']:
        bestHyperparams2019['ccNormalized_zscore']['recall']['value'] =interpolationZscore[R(rel=2)@10]
        bestHyperparams2019['ccNormalized_zscore']['recall']['alpha'] = alpha

    if interpolationZscore[nDCG@10] > bestHyperparams2019['ccNormalized_zscore']['ndcg']['value']:
        bestHyperparams2019['ccNormalized_zscore']['ndcg']['value'] =interpolationZscore[nDCG@10]
        bestHyperparams2019['ccNormalized_zscore']['ndcg']['alpha'] = alpha

    if interpolationZscore[RR(rel=2)@10] > bestHyperparams2019['ccNormalized_zscore']['rr']['value']:
        bestHyperparams2019['ccNormalized_zscore']['rr']['value'] =interpolationZscore[RR(rel=2)@10]
        bestHyperparams2019['ccNormalized_zscore']['rr']['alpha'] = alpha
    
    if fscoreInterpolationZscore > bestHyperparams2019['ccNormalized_zscore']['fscore']['value']:
        bestHyperparams2019['ccNormalized_zscore']['fscore']['value'] = fscoreInterpolationZscore
        bestHyperparams2019['ccNormalized_zscore']['fscore']['alpha'] = alpha
    
    if interpolationZscore[AP(rel=2)@10] > bestHyperparams2019['ccNormalized_zscore']['map']['value']:
        bestHyperparams2019['ccNormalized_zscore']['map']['value'] =interpolationZscore[AP(rel=2)@10]
        bestHyperparams2019['ccNormalized_zscore']['map']['alpha'] = alpha

In [ ]:
print(bestHyperparams2019['ccUnnormalized'])
print(bestHyperparams2019['ccNormalized_minimax'])
print(bestHyperparams2019['ccNormalized_zscore'])

## Validation of the RRF


In [ ]:
with open(
    "../testing/msmarco-test2019-queries.tsv",
    encoding="utf-8",
    newline=""
) as fp:
    queries = {q_id: q for q_id, q in csv.reader(fp, delimiter="\t")}
print(f"loaded {len(queries)} queries")

# define dictionary to keep track of best results for tested hyperparameters
bestHyperparams2019_rrf = {
    'rrf': {'precision': {'value': -1, 'eta1': 0, 'eta2': 0}, 
            'recall': {'value': -1, 'eta1': 0, 'eta2': 0}, 
            'fscore': {'value': -1, 'eta1': 0, 'eta2': 0}, 
            'rr': {'value': -1, 'eta1': 0, 'eta2': 0}, 
            'ndcg': {'value': -1, 'eta1': 0, 'eta2': 0}, 
            'map': {'value': -1, 'eta1': 0, 'eta2': 0}},
}

explored = set()

for i in tqdm(range(len(etas))):
    for j in range(len(etas)):
        if (etas[i],etas[j]) in explored:
            continue
        else:
            explored.add((etas[i],etas[j]))
            eta = etas[i]

        result2 = in_memory_index.get_scores_rrf(
            sparse_ranking_2019,
            queries,
            alpha=alpha,
            cutoff=None,
            early_stopping=False,
            eta = etas[i],
            eta2 = etas[j],
        )

        qrels = list(read_trec_qrels("../testing/2019qrels-pass.txt"))

        rrfRes = calc_aggregate([nDCG@10, RR(rel=2)@10,P(rel=2)@10,R(rel=2)@10,AP(rel=2)@10], qrels, result2[alpha].run)
        fscoreRRRes = (2*rrfRes[P(rel=2)@10]*rrfRes[R(rel=2)@10])/(rrfRes[P(rel=2)@10]+rrfRes[R(rel=2)@10])


        if rrfRes[P(rel=2)@10] > bestHyperparams2019_rrf['rrf']['precision']['value']:
            bestHyperparams2019_rrf['rrf']['precision']['value'] = rrfRes[P(rel=2)@10]
            bestHyperparams2019_rrf['rrf']['precision']['eta1'] = etas[i]
            bestHyperparams2019_rrf['rrf']['precision']['eta2'] = etas[j]


        if rrfRes[R(rel=2)@10] > bestHyperparams2019_rrf['rrf']['recall']['value']:
            bestHyperparams2019_rrf['rrf']['recall']['value'] = rrfRes[R(rel=2)@10]
            bestHyperparams2019_rrf['rrf']['recall']['eta1'] = etas[i]
            bestHyperparams2019_rrf['rrf']['recall']['eta2'] = etas[j]


        if rrfRes[nDCG@10] > bestHyperparams2019_rrf['rrf']['ndcg']['value']:
            bestHyperparams2019_rrf['rrf']['ndcg']['value'] = rrfRes[nDCG@10]
            bestHyperparams2019_rrf['rrf']['ndcg']['eta1'] = etas[i]
            bestHyperparams2019_rrf['rrf']['ndcg']['eta2'] = etas[j]

        if rrfRes[RR(rel=2)@10] > bestHyperparams2019_rrf['rrf']['rr']['value']:
            bestHyperparams2019_rrf['rrf']['rr']['value'] = rrfRes[RR(rel=2)@10]
            bestHyperparams2019_rrf['rrf']['rr']['eta1'] = etas[i]
            bestHyperparams2019_rrf['rrf']['rr']['eta2'] = etas[j]

        if fscoreRRRes > bestHyperparams2019_rrf['rrf']['fscore']['value']:
            bestHyperparams2019_rrf['rrf']['fscore']['value'] = fscoreRRRes
            bestHyperparams2019_rrf['rrf']['fscore']['eta1'] = etas[i]
            bestHyperparams2019_rrf['rrf']['fscore']['eta2'] = etas[j]

        if rrfRes[AP(rel=2)@10] > bestHyperparams2019_rrf['rrf']['map']['value']:
            bestHyperparams2019_rrf['rrf']['map']['value'] = rrfRes[AP(rel=2)@10]
            bestHyperparams2019_rrf['rrf']['map']['eta1'] = etas[i]
            bestHyperparams2019_rrf['rrf']['map']['eta2'] = etas[j]

In [ ]:
for key in bestHyperparams2019_rrf['rrf']:
    print(key, bestHyperparams2019_rrf['rrf'][key])

# Testing hyperparameters found from validation on 2020 set

## Testing the convex combination parameters


In [ ]:
with open(
    "../testing/msmarco-test2020-queries.tsv",
    encoding="utf-8",
    newline=""
) as fp:
    queries = {q_id: q for q_id, q in csv.reader(fp, delimiter="\t")}
print(f"loaded {len(queries)} queries")


for key in bestHyperparams2019.keys():
    print('\n'+'TESTING RESULTS FOR BEST VALIDATION METRICS FOR {}'.format(key))
    for metric in bestHyperparams2019[key].keys():
        # print(f"best {metric} for {key} is {bestHyperparams2019[key][metric]['value']} with alpha {bestHyperparams2019[key][metric]['alpha']}")
    
        alpha = bestHyperparams2019[key][metric]['alpha']

        result = in_memory_index.get_scores(
            sparse_ranking_2020,
            queries,
            alpha=alpha*1.0,
            cutoff=None,
            early_stopping=False,
            normalise= False if key == 'ccUnnormalized' else True
        )
        qrels = list(read_trec_qrels("../testing/2020qrels-pass.txt"))
        print('using best alpha={} for {}'.format(alpha, metric))
        interpolation2 = calc_aggregate([nDCG@10, RR(rel=2)@10,P(rel=2)@10,R(rel=2)@10,AP(rel=2)@10], qrels, result[alpha].run)
        fscoreInterpolation2 = (2*interpolation2[P(rel=2)@10]*interpolation2[R(rel=2)@10])/(interpolation2[P(rel=2)@10]+interpolation2[R(rel=2)@10])
        print(
            f"Interpolation",
            interpolation2,
            'f-score:{}'.format(fscoreInterpolation2)
        )

## Testing RRF parameters

In [ ]:
with open(
    "../testing/msmarco-test2020-queries.tsv",
    encoding="utf-8",
    newline=""
) as fp:
    queries = {q_id: q for q_id, q in csv.reader(fp, delimiter="\t")}
print(f"loaded {len(queries)} queries")


for key in bestHyperparams2019_rrf.keys():
    print('\n'+'TESTING RESULTS FOR BEST VALIDATION METRICS FOR {}'.format('normalized' if key == 'ccNormalized' else 'unnormalized'))
    for metric in bestHyperparams2019_rrf[key].keys():
        # print(f"best {metric} for {key} is {bestHyperparams2019[key][metric]['value']} with alpha {bestHyperparams2019[key][metric]['alpha']}")
        eta1 = bestHyperparams2019_rrf[key][metric]['eta1']
        eta2 = bestHyperparams2019_rrf[key][metric]['eta2']
        alpha = 0.0
        result = in_memory_index.get_scores_rrf(
            sparse_ranking_2020,
            queries,
            alpha=alpha,
            cutoff=None,
            early_stopping=False,
            eta = eta1,
            eta2 = eta2
        )
        qrels = list(read_trec_qrels("../testing/2020qrels-pass.txt"))
        print('using best eta1={} and eta2={} for {}'.format(eta1, eta2, metric))
        interpolation2 = calc_aggregate([nDCG@10, RR(rel=2)@10,P(rel=2)@10,R(rel=2)@10,AP(rel=2)@10], qrels, result[alpha].run)
        fscoreInterpolation2 = (2*interpolation2[P(rel=2)@10]*interpolation2[R(rel=2)@10])/(interpolation2[P(rel=2)@10]+interpolation2[R(rel=2)@10])
        print(
            f"RRF",
            interpolation2,
            'f-score:{}'.format(fscoreInterpolation2)
        )